In [ ]:
import requests
from PIL import Image, ImageDraw, ImageFont
import io
import os
import base64
import random

# Replace with actual API keys or endpoints
LLAVA_API_ENDPOINT = "https://supplement-convenient-scores-forgotten.trycloudflare.com/llava/generate"
LLAVA_API_KEY = "Mklm9M70UL39aQhBLiDSDM2yv1V73uXWMA-GBN1LBDU"
FLUX_API_ENDPOINT = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
FLUX_API_KEY = "calm-bold-tree-bd1b1c8bc141403231e7f0a8"

# API headers
LLAVA_HEADERS = {
    "Authorization": f"Bearer {LLAVA_API_KEY}",
    "Content-Type": "application/json"
}
FLUX_HEADERS = {
    "Authorization": f"Bearer {FLUX_API_KEY}",
    "Content-Type": "application/json"
}

# Helper function: Generate marketing text using LLAVA API
def generate_text(prompt, tone="friendly", guidelines=None):
    formatted_prompt = f"Focus 80% of the response on the following user prompt and 20% on the guidelines. Give relevant hastags at the end of the response. User Prompt: {prompt}. Guidelines: {guidelines}" if guidelines else prompt
    payload = {
        "inputs": f"<|im_start|>system\nAnswer concisely with only what is asked.<|im_end|><|im_start|>user\n{formatted_prompt}<|im_end|><|im_start|>assistant\n",
        "parameters": {"max_new_tokens": 500, "temperature": 0.7}
    }
    response = requests.post(LLAVA_API_ENDPOINT, json=payload, headers=LLAVA_HEADERS)
    response.raise_for_status()
    generated_text = response.json().get("generated_text", "No response text available")

    return f"{generated_text}"

# Helper function: Generate an image using Flux API
def generate_image(description):
    payload = {
        "prompt": description,
        "img_size": 1024,
        "guidance_scale": 7.5,
        "num_inference_steps": 50
    }
    response = requests.post(FLUX_API_ENDPOINT, json=payload, headers=FLUX_HEADERS)
    response.raise_for_status()
    image_data = response.content  # Directly get the binary content of the image
    return Image.open(io.BytesIO(image_data))

# Helper function: Add text to an image
def add_text_to_image(image, text, position=(10, 10), font_path=None, font_size=40):
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype(font_path, font_size) if font_path else ImageFont.load_default()
    draw.text(position, text, fill="white", font=font)
    return image

# Helper function: Add a logo to the image
def add_logo_to_image(image, logo_path, position=(10, 10), logo_size=(100, 100)):
    logo = Image.open(logo_path).convert("RGBA")  # Convert logo to RGBA to handle transparency
    logo = logo.resize(logo_size)

    # Ensure the generated image is in RGBA mode for transparency support
    if image.mode != "RGBA":
        image = image.convert("RGBA")

    # Paste the logo onto the image, ensuring transparency is handled correctly
    image.paste(logo, position, logo)  # The third argument (logo) acts as a mask for transparency
    return image

# Main function: Create a marketing post
def create_marketing_post(prompt, platform, logo_path=None, guidelines=None):
    print(f"Generating content for {platform}...")

    # Generate text based on platform and guidelines
    tone = "friendly" if platform == "Instagram" else "professional"
    text = generate_text(prompt, tone, guidelines)

    # Generate corresponding visual
    description = f"{prompt} for a {platform} post"
    image = generate_image(description)

    # Add text overlay for Instagram
    if platform == "Instagram":
        image = add_text_to_image(image, text, position=(20, 20))

    # Add logo to the image if provided
    if logo_path:
        image = add_logo_to_image(image, logo_path)

    # Add guidelines text if provided
    if guidelines:
        image = add_text_to_image(image, guidelines, position=(10, 900), font_size=30)

    return text, image

# Example usage
if __name__ == "__main__":
    try:
        # Input prompt, brand details, and additional parameters
        prompt = "Imagine kayaking down a snowy mountain, where the river has frozen into slick, powdery paths that wind across steep slopes. The thrill of navigating these snow-covered terrains on a kayak provides a unique challenge, blending the art of kayaking with winter sports."
        guidelines = "A kayak club should ensure safety is always the top priority by maintaining proper equipment, organizing regular safety briefings, and setting guidelines for river conditions and weather forecasts. Membership requirements should emphasize skill levels to ensure appropriate participation, while promoting a welcoming environment for enthusiasts of all abilities. The club must also encourage environmental responsibility, ensuring that members respect waterway conservation efforts and follow guidelines for minimizing their ecological impact during outdoor activities."

        # Ask user for input (logo path)
        logo_path = input("Enter the path to the logo image (or leave empty): ").strip()

        # Directory to save outputs
        output_dir = "marketing_posts"
        os.makedirs(output_dir, exist_ok=True)

        # Generate and save content for each platform
        platforms = ["Instagram", "LinkedIn"]
        for platform in platforms:
            for i in range(1):  # Generate 3 posts for each platform
                text, image = create_marketing_post(prompt, platform, logo_path=logo_path, guidelines=guidelines)

                # Save image
                image_path = os.path.join(output_dir, f"{platform}_post_{i+1}.png")
                image.save(image_path)
                print(f"Image saved: {image_path}")

                # Save text
                text_path = os.path.join(output_dir, f"{platform}_post_{i+1}.txt")
                with open(text_path, "w") as text_file:
                    text_file.write(text)
                print(f"Text saved: {text_path}")

        print("Marketing posts created successfully!")
    except Exception as e:
        print(f"Error: {e}")
